# Поиск вырубок. Инициализация.

Описывается инициализация данных. Основные моменты:

1. Основное хранилище данных - GRASS GIS.
2. В БД GRASS [импортируются растры](#import_landsat) Landsat 8.
3. Для каждой сцены [производится расчет](#scene_import) отражательной способности (i.landsat.toar).
4. Импортированы контура рубок за 2015-2016гг.

## Вспомогательные функции и модули

Импортируем вспомогательные модули.

In [8]:
import os
import glob
import tempfile
import shutil

import utilites
reload(utilites)

<module 'utilites' from 'utilites.pyc'>

Импортируем функции для выполнения рутинных действий.

Функция распаковки архивов. Параметры: data_file -- архив, extract_dir -- каталог, в который производится распаковка.

In [2]:
from utilites import unpack

Функции, чтобы не запутаться в том, как формируется название location (будут использоваться в разных местах кода):

In [3]:
from utilites import (
    get_grassdata_path,
    get_location_name,
    get_location_path,
)

print get_location_path()

/home/klsvd/GRASSDATA/LANDSAT


Функции для работы с файлами сцены:

In [4]:
from utilites import (
    filename_to_bandname,
    get_raster_list,
    find_meta
)

## Импорт данных

Сначала создадим объект GRASS, через который будем выполнять запросы.

In [9]:
from grasslib import GRASS

grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name()
)

### Импорт Landsat<a id='import_landsat'></a>

Пусть сцены лежат в подкаталоге Data домашнего каталога. Создадим переменную SCENE_DIR, храняющую пути к этому каталогу:

In [5]:
HOME_DIR = os.getenv("HOME")
SCENE_DIR = os.path.join(HOME_DIR, 'Data')

print SCENE_DIR

/home/klsvd/Data


Получим список сцен:

In [6]:
scenes = glob.glob(os.path.join(SCENE_DIR, '*gz'))

print scenes

['/home/klsvd/Data/LC81120272015365LGN00.tar.gz', '/home/klsvd/Data/LC81120282015141LGN00.tar.gz', '/home/klsvd/Data/LC81120282016032LGN01.tar.gz', '/home/klsvd/Data/LC81120272016032LGN01.tar.gz', '/home/klsvd/Data/LC81120282015317LGN00.tar.gz', '/home/klsvd/Data/LC81120282015237LGN00.tar.gz', '/home/klsvd/Data/LC81120282016016LGN00.tar.gz', '/home/klsvd/Data/LC81120272015157LGN00.tar.gz', '/home/klsvd/Data/LC81120272015189LGN00.tar.gz', '/home/klsvd/Data/LC81120282015189LGN00.tar.gz', '/home/klsvd/Data/LC81120272015333LGN00.tar.gz', '/home/klsvd/Data/LC81120282015365LGN00.tar.gz']


#### Создание БД GRASS <a id='create_location'></a>

Если область GRASS для работы уже создана, можно сразу переходить к [импорту сцен](#scene_import). Если нет -- создадим в домашнем каталоге каталог GRASSDATA и внутри него LOCATION для хранения сцен. Для этого распакуем во временный каталог первую сцену, прочитаем из нее геоданные и создадим соответствующий LOCATION.

In [7]:
scene = scenes[0]
location = get_location_path()

try:
    temp_dir = tempfile.mkdtemp()
    print temp_dir
    if unpack(scene, temp_dir):
        print 'Unpacked'
    else:
        print "Can't unpack scene"
    
    # Возьмем первый попавшийся канал и прочитаем из него геоданные
    file_list = get_raster_list(temp_dir)
    geofile = os.path.join(temp_dir, file_list[0])
    print geofile
    
    cmd = 'grass70 -text -e -c %s %s' % (geofile, location)
    print cmd
    
    exitcode = os.system(cmd)
    if exitcode != 0:
        print "Can't create location"
finally:
    shutil.rmtree(temp_dir)
    print 'Removed'


/tmp/tmpFUyJpE
Unpacked
/tmp/tmpFUyJpE/LC81120272015365LGN00_B2.TIF
grass70 -text -e -c /tmp/tmpFUyJpE/LC81120272015365LGN00_B2.TIF /home/klsvd/GRASSDATA/LANDSAT
Removed


#### Импорт сцен <a id='scene_import'></a>

Импортируем сцены и рассчитаем спектральную отражательную способность (ToAR Reflectance):

1. Распаковываем сцену во временный каталог.
2. Импортируем каналы сцены.
3. Устанавливаем значения null (нет данных).
4. Рассчитываем отражательную способность.
5. Удаляем импортированные растры, хранящие DN (уже не нужны, есть растры отражательной способности).
6. Удаляем временный каталог, содержащий распакованные сцены.

In [8]:
# Определим для удобства функцию
def import_scene(grs, scene):
    try:
        # Шаг 1
        temp_dir = tempfile.mkdtemp()
        if unpack(scene, temp_dir):
            print 'Unpacked'
        else:
            print "Can't unpack scene"

        rasters = get_raster_list(temp_dir)
        for rst in rasters:
            # Шаг 2
            band_name = filename_to_bandname(rst)
            grs.grass.run_command('r.in.gdal', input=rst, output=band_name, overwrite=True, flags='e')

            # Шаг 3
            grs.grass.run_command('r.null', map=band_name, setnull=0)


        # Шаг 4
        indx = band_name.rindex('B')    # Названия всех каналов устроены одинаково
        prefix = band_name[:indx + 1]   # поэтому для вычисления префикса можно использовать любой
        metafile = find_meta(temp_dir)
        grs.grass.run_command('i.landsat.toar', input=prefix, output='toar_' + prefix,
                metfile=metafile, sensor='oli8', overwrite=True)
        

        # Шаг 5
        grs.grass.run_command('g.remove', type='rast', 
                              pattern='%s*' % (prefix, ), 
                              exclude="*BQA",
                              flags='f')


    finally:
        # Шаг 6
        shutil.rmtree(temp_dir)
        print 'Removed'
        
        
for scene in scenes:
    import_scene(grs, scene)


Unpacked
Removed
Unpacked
Removed
Unpacked
Removed
Unpacked
Removed
Unpacked
Removed
Unpacked
Removed
Unpacked
Removed
Unpacked
Removed
Unpacked
Removed
Unpacked
Removed
Unpacked
Removed
Unpacked
Removed


#### Группы растров

Создадим группы растров для удобства работы (многие GRASSовские модули обработки изображений "хотят" на входе группу).

In [9]:
rasters = grs.grass.list_strings("rast")
print rasters[:10]

['LC81120272015157LGN00_BQA@PERMANENT', 'LC81120272015189LGN00_BQA@PERMANENT', 'LC81120272015333LGN00_BQA@PERMANENT', 'LC81120272015365LGN00_BQA@PERMANENT', 'LC81120272016032LGN01_BQA@PERMANENT', 'LC81120282015141LGN00_BQA@PERMANENT', 'LC81120282015189LGN00_BQA@PERMANENT', 'LC81120282015237LGN00_BQA@PERMANENT', 'LC81120282015317LGN00_BQA@PERMANENT', 'LC81120282015365LGN00_BQA@PERMANENT']


In [10]:
vectors = grs.grass.list_strings("vect")
print vectors

[]


In [11]:
group_names = set([name.rpartition('_')[0] for name in rasters])
group_names = list(group_names)

for gname in group_names:
    rnames = grs.grass.read_command('g.list', type='raster', 
                              pattern=gname+'*', 
                              separator=',')
    rnames = rnames.strip()
    grs.grass.run_command('i.group', group=gname,
                   input=rnames)

groups = grs.grass.read_command('g.list', type='group')
print groups

LC81120272015157LGN00
LC81120272015189LGN00
LC81120272015333LGN00
LC81120272015365LGN00
LC81120272016032LGN01
LC81120282015141LGN00
LC81120282015189LGN00
LC81120282015237LGN00
LC81120282015317LGN00
LC81120282015365LGN00
LC81120282016016LGN00
LC81120282016032LGN01
toar_LC81120272015157LGN00
toar_LC81120272015189LGN00
toar_LC81120272015333LGN00
toar_LC81120272015365LGN00
toar_LC81120272016032LGN01
toar_LC81120282015141LGN00
toar_LC81120282015189LGN00
toar_LC81120282015237LGN00
toar_LC81120282015317LGN00
toar_LC81120282015365LGN00
toar_LC81120282016016LGN00
toar_LC81120282016032LGN01



In [12]:
example = grs.grass.read_command('i.group', group=gname, flags='lg')
print example

toar_LC81120282016016LGN00_B1@PERMANENT
toar_LC81120282016016LGN00_B10@PERMANENT
toar_LC81120282016016LGN00_B11@PERMANENT
toar_LC81120282016016LGN00_B2@PERMANENT
toar_LC81120282016016LGN00_B3@PERMANENT
toar_LC81120282016016LGN00_B4@PERMANENT
toar_LC81120282016016LGN00_B5@PERMANENT
toar_LC81120282016016LGN00_B6@PERMANENT
toar_LC81120282016016LGN00_B7@PERMANENT
toar_LC81120282016016LGN00_B8@PERMANENT
toar_LC81120282016016LGN00_B9@PERMANENT



### Импорт рубок<a id='import_train'></a>

Пусть данные по рубкам лежат в каталоге "Data/ALARM/trainings2016".

In [15]:
train_file = os.path.join(
    HOME_DIR,'Data/ALARM/trainings2016', 'training2015-2016_winter.shp')

In [16]:
grs.grass.run_command(
    'v.in.ogr', input=train_file,
    output='train15_16', overwrite=True
)

0